<img align=right src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150>
<img src=images/sglogo.png width=50 align="left">

# Cloud Authentication Methods
<img src="images/cloud-auth.png" width=600>  


Vault supports many different authentication methods, suitable for human and applications (e.g. AppRole, AWS/Azure/Google, K8s).
Most Vault auth methods need to be explicitly enabled.  Each method has a default path.


**Prerequisites:** 
You need to have Vault up and running

`VAULT_UI=true VAULT_REDIRECT_ADDR=http://127.0.0.1:8200 vault server -log-level=trace -dev -dev-root-token-id=root -dev-listen-address=127.0.0.1:8200 -dev-ha -dev-transactional`

In [ ]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_SKIP_VERIFY=true

In [ ]:
vault login root

In [ ]:
vault status; vault version

In [ ]:
vault list auth/userpass/users

---
# AWS Auth method


There are two authentication types in the AWS auth method:  `iam` and `ec2`.  

The aws auth method provides an automated mechanism to retrieve Vault token for IAM principals and AWS EC2 instances.   
This method does not require manual first-deploying or profisioning security-sensitive credentials.


### Setup AWS auth config

In [ ]:
export VAULT_ADDR=http://127.0.0.1:8200

In [ ]:
vault login root

In [ ]:
vault auth enable aws

In [ ]:
source ~/.zshrc; echo $AWS_ACCESS_KEY_ID; aws sts get-caller-identity

In [ ]:
# tbagio profile in ~/.aws/credentials
#
vault write auth/aws/config/client \
  secret_key=$AWS_SECRET_ACCESS_KEY \
  access_key=$AWS_ACCESS_KEY_ID 

In [ ]:
vault read auth/aws/config/client

&nbsp;   
### IAM Authentication type. 
<img src="images/iam-auth-flow.png" width=400>  

1. The AWS User makes a request to Vault and includes its IAM credential.   

2. Vault verifies that the User is valid  using the public AWS API endpoint.   

3. Provided all steps are successful, Vault returns the initial Vault token to the AWS User. This token is mapped to any configured policies based on the instance metadata.   


In [ ]:
vault write auth/aws/role/dev-role-iam \
   auth_type=iam \
   bound_iam_principal_arn="arn:aws:iam::451256726182:*"\
   policies=dynamic-policy,prod,dev ttl=5m

In [ ]:
vault write auth/aws/config/client iam_server_id_header_value=vault.example.com

In [ ]:
vault read auth/aws/role/dev-role-iam

### Demo: Login using AWS IAM information

&nbsp;   
### IAM Authentication type. 
<img src="images/iam-auth-flow.png" width=400>  

1. The AWS User makes a request to Vault and includes its IAM credential.   

2. Vault verifies that the User is valid  using the public AWS API endpoint.   

3. Vault returns the initial Vault token to the AWS User. This token is mapped to any configured policies based on the instance metadata.   


In [22]:
AWS_ACCESS_KEY_ID=$(cat /tmp/aws_id); AWS_SECRET_ACCESS_KEY=$(cat /tmp/aws_secret);  \
aws sts get-caller-identity

{h=
    "UserId": "AIDAWSEH7GKTD4E53KCTO",
    "Account": "451256726182",
    "Arn": "arn:aws:iam::451256726182:user/raymond"
}


In [23]:
# raymond
vault login -method=aws header_value=vault.example.com \
  role=dev-role-iam \
  aws_access_key_id=$(cat /tmp/aws_id) \
  aws_secret_access_key=$(cat /tmp/aws_secret)

Success! You are now authenticated. The token information displayed below
is already stored in the token helper. You do NOT need to run "vault login"
again. Future Vault requests will automatically use this token.

Key                      Value
---                      -----
token                    s.vqyJinyWay6RYnBWnYrsi7mp
token_accessor           Eerh8Ufv3elwNb6STZ5KybO4
token_duration           5m
token_renewable          true
token_policies           ["default" "dev" "dynamic-policy" "prod"]
identity_policies        []
policies                 ["default" "dev" "dynamic-policy" "prod"]
token_meta_account_id    451256726182
token_meta_auth_type     iam
token_meta_role_id       6503c4c8-1d37-4a1a-1c66-68db7bcc025f


### and get a secret

In [24]:
VAULT_TOKEN=s.vqyJinyWay6RYnBWnYrsi7mp vault  kv get secret1/web01

====== Data ======
Key         Value
---         -----
ip          192.168.1.191
password    passw0rd123
username    arthur


In [25]:
VAULT_TOKEN=s.vqyJinyWay6RYnBWnYrsi7mp  vault read mdb1/creds/manager

Key                Value
---                -----
lease_id           mdb1/creds/manager/g1dGxmdJgfwBpLd0m9whtvpn
lease_duration     3m
lease_renewable    true
password           u&%*QMD6
username           v_mana_qUfBrHF5E


In [26]:
docker exec -it workshop-mysql mysql -u vaultadmin -p'vadmin' -e "select user from mysql.user"; 

mysql: [Warning] Using a password on the command line interface can be insecure.
+------------------+
| user             |
+------------------+
| root             |
| v_mana_qUfBrHF5E |
| vault            |
| vaultadmin       |
| healthchecker    |
| mysql.session    |
| mysql.sys        |
| root             |
+------------------+


In [28]:
docker exec -it workshop-mysql mysql -u v_mana_qUfBrHF5E -p'u&%*QMD6' \
  -e "use my_app; select * from my_app.test"

mysql: [Warning] Using a password on the command line interface can be insecure.
+-------+
| score |
+-------+
|    10 |
|     7 |
+-------+


---
### EC2 Authentication Method
<img src="images/aws-ec2-auth-flow.png" width=500>   

1. An AWS EC2 instance fetches its AWS Instance Identity Document from the EC2 Metadata Service.   

2. The AWS EC2 instance makes a request to Vault and includes Instance Identity Document.   

3. Vault verifies that the instance is currently running using the public EC2 API endpoint.   

4. Provided all steps are successful, Vault returns the initial Vault token to the EC2 instance. This token is mapped to any configured policies based on the instance metadata.   


**Configure Vault for EC2 Authentication type**

In [ ]:
VAULT_TOKEN=root vault write auth/aws/role/dev-role-ec2 \
   auth_type=ec2 \
   bound_ami_id=ami-fce3c696 \
   policies=prod,dev max_ttl=5m

In [ ]:
vault read auth/aws/role/dev-role-ec2

---
# Github Authentication method

### Setup Vault-Github integration

In [ ]:
export VAULT_ADDR=http://127.0.0.1:8200
vault login root

In [ ]:
vault auth enable github

### Configure Vault to talk to Github

In [ ]:
vault write auth/github/config organization=hashicorp

### Map users/teams of that Github organization to policies in Vault.  
Team names must be "slugified"

In [ ]:
vault write auth/github/map/teams/team-se value=static-policy

In [ ]:
vault write auth/github/map/users/tiobagio value=dynamic-policy

### Login using Github token
The `github` auth method can be used to authenticate with Vault using a Github personal access token.  This method of authentication is most useful for operators or developers usign Vault directly.

In [ ]:
MY_GITHUB_TOKEN=$(cat /tmp/githubtoken)
vault login -method=github token=$MY_GITHUB_TOKEN

**Using cURL**

In [ ]:
MY_GITHUB_TOKEN=$(cat /tmp/githubtoken)
curl -s --request POST --data " {\"token\": \"$MY_GITHUB_TOKEN\"} " \
    http://127.0.0.1:8200/v1/auth/github/login  | jq

---
# Okta Auth method

### Configure Vault to talk to Okta

In [ ]:
export VAULT_ADDR=http://127.0.0.1:8200
vault login root
vault auth enable okta

In [ ]:
vault write auth/okta/config \
  base_url="okta.com" \
  org_name="hashicorp" \
  token_policies="static-policy"

**Note:**  
Anyone who successfully authenticates via Okta who's a member of Hashicorp organization will receive a Vault token with "static-policy" attached.   
Furthermore for a specific user, he/she gets additional policies like "dynamic-policy".

In [ ]:
vault write auth/okta/users/tio@hashicorp.com policies="dynamic-policy,other-policy"

### Login using Okta method

In [ ]:
## you'll be prompted for password
vault login -method=okta username=xyz@hashi.com

**Using cURL**

In [ ]:
MY_PASSWORD=$(cat /tmp/mypassword)
curl -s --request POST --data " {\"password\": \"$MY_PASSWORD\"} " \
    http://127.0.0.1:8200/v1/auth/okta/login/tio@hashicorp.com

---
### TODO:
- how to set AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS for a certain profile
- use "bound_iam_principal_arn=arn:aws:iam:::user/\ "
- use "bound_iam_role_arn=arn:aws:iam::451256726182:role/example-role\ "

---
# Azure Auth 

In [ ]:
vault login root
vault auth enable azure

In [ ]:
vault write auth/azure/config \
    resource=https://vault.hashicorp.com \
    tenant_id= \
    client_id= \
    client_secret=

In [ ]:
vault write auth/azure/role/dev-role \
    policies="prod,dev" \
    bound_subscription_ids=6a1d5988- \
    bound_resource_groups=vault

### Login

In [ ]:
vault write auth/azure/login \
   role="dev-role" \
   jwt="eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9..." \
   subscription_id="12345-..." \
   resource_group_name="test-group" \
   vm_name="test-vm"

In [ ]:
vault write auth/azure/login \
   role="dev-role" \
   jwt="$(curl -s 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=2018-02-01&resource=https%3A%2F%2Fvault.hashicorp.com%2F' -H Metadata:true | jq -r '.access_token')" \
   subscription_id=$(curl -s -H Metadata:true "http://169.254.169.254/metadata/instance?api-version=2017-08-01" | jq -r '.compute | .subscriptionId')  \
   resource_group_name=$(curl -s -H Metadata:true "http://169.254.169.254/metadata/instance?api-version=2017-08-01" | jq -r '.compute | .resourceGroupName') \
   vm_name=$(curl -s -H Metadata:true "http://169.254.169.254/metadata/instance?api-version=2017-08-01" | jq -r '.compute | .name')

---
# Google Cloud Auth Method
The `gcp` auth method allows Google Cloud Platform entities to authenticate to Vault.  
Vault treats Google Cloud as a trusted third party and verifies authenticating entities against the Google Cloud APIs.
  
This backend allow for authentication of:
* IAM service accounts
* GCE (compute engine) instances

It does not support authenticating arbitrary Google/G Suite users or generic OAuth against Google.

### Setup

In [ ]:
vault login root
vault auth enable gcp

In [ ]:
vault write auth/gcp/config credentials=@/path/to/credentials.json

**Create a role**  
For `iam` type role.  Note that `bound_service_account` is only required for `iam`-type roles

In [ ]:
vault write auth/gcp/role/my-iam-role \
   type="iam" \
   policies="dev, prod" \
   bound_service_accounts="my-service@my-project.iam.gserviceaccount.com"

For `gce` type role

In [ ]:
vault write auth/gcp/role/my-gce-role \
   type="gce" \
   policies="dev, prod" \
   bound_projects="my-prject1,my-project2" \
   bound_zones="us-east1-b" \
   bound_labels="fooLbar, zip:zap" \
   bound_service_accounts="my-service@my-project.iam.gserviceaccount.com"

### Login/Authenticate

In [ ]:
vault write -field=token auth/gcp/login \
  role="my-role" \
  jwt="..." 

Vault includes a CLI helper that obtains a signed JWT locally and sends the request to Vault.  This helper is only for IAM-type roles.

In [ ]:
vault login -method=gcp \
   role="my-role" \
   service_account="authenticating-account@my-project.iam.gserviceaccount.com" \
   project="my-project" \
   jwt_exp="15m" \
   credentials=@path/to/signer/credentials.json

---
### View Vault via a browser
Open a web browser and visit http://127.0.0.1:8200/ui/vault
Use **TOKEN** and **Sign in**

&nbsp;

---
#### Thank you.
<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=100 align="left">